# Set Up for Project Imports


In [ ]:
import sys
from pathlib import Path

In [ ]:
source_directory = Path.cwd()
ROOT = source_directory.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [ ]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
from data.api import UcIrvineAPI, UcIrvineDatasetIDs
import data.wrangling_utils
import pandas
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    r2_score,
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    median_absolute_error,
)

In [ ]:
pandas.set_option('display.max_colwidth', None)  # show all text in cells
# pandas.set_option("display.max_rows", 100_000)
pandas.options.mode.copy_on_write = True
pandas.set_option('display.float_format', lambda x: '%.2f' % x)

# UcIrvine Data

In [ ]:
uci = UcIrvineAPI.fetch_dataset(repo_id=UcIrvineDatasetIDs.Apartment_For_Rent_Classified.value)
uci_df: pandas.DataFrame = uci.data.original.reset_index()

In [ ]:
uci_df.describe()

In [ ]:
uci_df.shape

# Data Cleaning

In [ ]:
clean_uci_df: pandas.DataFrame = data.wrangling_utils.clean(uci_df)

In [ ]:
clean_uci_df.info()

# Exploratory Data Analysis


#### Top 5 Listing Sources

In [ ]:
sources_df = clean_uci_df.groupby(['source']).size().reset_index(name='count').sort_values(by='count', ascending=False)
sources_df.head(5)

#### Distribution of Count of Listings per State

In [ ]:
clean_uci_df.sort_values(by='state', inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 12))
ax = sns.countplot(data=clean_uci_df, x='state', width=0.5)

for container in ax.containers:
    ax.bar_label(container)

plt.xticks(rotation=90)
plt.tight_layout()

#### Descriptive Statistics of all listings

In [ ]:
clean_uci_df.describe()

An apartment with 9 bedrooms, a monthly cost of $120,000, and 50,000 square feet? Is this actually an apartment? Let's examine the listings by combinations of bedrooms and bathrooms, and distributions of prices and square footage.. It will give insight on the type of properties in our data.

## Removing Noise

In [ ]:
filtered_df = clean_uci_df.groupby(['bedrooms', 'bathrooms']).size().reset_index(name='count')
filtered_df

It appears studios, townhomes, houses, mansions, or penthouses are also in the data. Some properties don't even have a bedroom? Other properties have 4+ bedrooms and bathrooms? That doesn't make sense for a typical apartment.

In [ ]:
min_bedrooms = 1
max_bedrooms = 3
filtered_df = clean_uci_df[(clean_uci_df['bedrooms'] >= min_bedrooms) & (clean_uci_df['bedrooms'] <= max_bedrooms) & (clean_uci_df['bathrooms'] <= 3)]
filtered_df.describe()

Why is 3 bed and 3 bath priced at $33,0000? That sees overpriced even for a luxury apartment. Let's examine the distribution of listing prices for our data.

In [ ]:
price_distribution_df = filtered_df.groupby(['price']).size().reset_index(name='count')
sns.histplot(price_distribution_df['price'], bins=50)
plt.xlim(price_distribution_df['price'].min(), price_distribution_df['price'].max())
plt.show()

The distribution of prices is unimodal, and it is strongly right skewed where most the data is centered around 2,000-3,0000 USD. There are extreme outliers of 10,000-30,000. Maybe luxurious apartments/penthouse are causing this.

In [ ]:
min_price = 500
max_price = 10000
filtered_df = filtered_df[(filtered_df['price'] >= min_price) & (filtered_df['price'] <= max_price)]
filtered_df.describe()

Now let's examine the distribution of prices again.

In [ ]:
price_distribution_df = filtered_df.groupby(['price']).size().reset_index(name='count')
sns.histplot(price_distribution_df['price'], bins=50)
plt.show()

The distribution of the listing prices is still right skewed but we got rid of some noise. Now let's examine the distribution of the square footage of the listings since ~5,000 square foot apartment still seems large for a max 3 bedroom and 3 bathroom apartment.

In [ ]:
sqft_distribution_df = filtered_df.groupby(['square_feet']).size().reset_index(name='count')
sns.histplot(sqft_distribution_df['square_feet'], bins=24)
plt.xlim(sqft_distribution_df['square_feet'].min(), sqft_distribution_df['square_feet'].max())
plt.show()

In [ ]:
max_sq_ft = 4000
filtered_df = filtered_df[(filtered_df['square_feet'] <= max_sq_ft)]
filtered_df.describe()

In [ ]:
sns.histplot(filtered_df['square_feet'], bins=24)
plt.xlim(filtered_df['square_feet'].min(), filtered_df['square_feet'].max())
plt.show()

In [ ]:
#select monthly price
cleaned_subset_df = clean_uci_df[(clean_uci_df['bedrooms'] >= min_bedrooms) & (clean_uci_df['bedrooms'] <= max_bedrooms) & (clean_uci_df['bathrooms'] <= 3) & (filtered_df['price'] >= min_price) & (filtered_df['price'] <= max_price) & (clean_uci_df['square_feet'] <= max_sq_ft) & (clean_uci_df['price_type'] == 'monthly') & (clean_uci_df['state'] is not None)]

#select only these columns
cleaned_subset_df = cleaned_subset_df[['bathrooms', 'bedrooms', 'price', 'square_feet', 'state', 'latitude', 'longitude', 'cityname']]

cleaned_subset_df = cleaned_subset_df.dropna()

In [ ]:
cleaned_subset_df.describe()

In [ ]:
cleaned_subset_df.shape

In [ ]:
percent_change = (1.0 - (cleaned_subset_df.shape[0] / uci_df.shape[0])) * 100
print(f"Percent change {percent_change:.3f}%")

## More Skewness Analysis

In [ ]:
cleaned_subset_df[['price', 'square_feet']].skew()

| Feature       | Skew     | Interpretation            | Action               |
| ------------- |----------|---------------------------|----------------------|
| `price`       | **2.28** | **Strongly right-skewed** | Should transform     |
| `square_feet` | **1.17** | **Strongly right-skewed** | Should transform     |


Most the data is small to large apartments prices around $1,500 - 3,000 but a handful of extremely underpriced/overpriced apartments are skewing the data. So we will perform a log transformation to reduce their influence.

In [ ]:
cleaned_subset_df['price_log'] = np.log1p(cleaned_subset_df['price'])
cleaned_subset_df['square_feet_log'] = np.log1p(cleaned_subset_df['square_feet'])
cleaned_subset_df[['price_log', 'square_feet_log']].skew()

| Feature       | Skew              | Interpretation         |
|---------------|-------------------|-------------------------|
| `price`       | **2.28 -> 0.36**  | **Weakly right-skewed** |
| `square_feet` | **1.17 -> -0.21** | **Weakly left-skewed**  |

In [ ]:
sns.histplot(cleaned_subset_df['price_log'], bins=24)
plt.xlim(cleaned_subset_df['price_log'].min(), cleaned_subset_df['price_log'].max())
plt.show()

In [ ]:
sns.histplot(cleaned_subset_df['square_feet_log'], bins=24)
plt.xlim(cleaned_subset_df['square_feet_log'].min(), cleaned_subset_df['square_feet_log'].max())
plt.show()

In [ ]:
grid = sns.pairplot(cleaned_subset_df[['bathrooms', 'bedrooms', 'price_log', 'square_feet_log']], kind='scatter', corner=True)

There only seems a weak linear relationship between log_price and square_feet_log.

In [ ]:
correlation = cleaned_subset_df[['bathrooms', 'bedrooms', 'price_log', 'square_feet_log']].corr(numeric_only=True)

In [ ]:
correlation

In [ ]:
plt.figure(figsize=(8, 6))
plt.title("Correlation Heatmap", fontsize=14)
sns.heatmap(correlation, annot=True, linewidths=0.5, cmap='mako')
plt.show()

### Variable Types

- Discrete / ordinal: bedrooms, bathrooms — integer counts
- Continuous: price_log, square_feet_log — continuous and normalized
- We used Pearson correlation, the relationships involving discrete counts are approximate linear associations, not strict parametric correlations — but they’re still informative here since the discrete values are ordered and range reasonably (0–9).


| Pair                            | Correlation | Interpretation                                                                                                                                            |
| ------------------------------- |-------------| --------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **bathrooms ↔ bedrooms**        | **0.65**    | Strong positive relationship — homes with more bedrooms usually have more bathrooms.                                                                      |
| **bathrooms ↔ square_feet_log** | **0.68**    | Strong positive correlation — larger houses naturally have more bathrooms.                                                                                |
| **bedrooms ↔ square_feet_log**  | **0.66**    | Same strong pattern — larger homes have more bedrooms.                                                                                                    |
| **price_log ↔ square_feet_log** | **0.36**    | Moderate positive relationship — price generally rises with size, but not perfectly (other factors matter).                                               |
| **price_log ↔ bathrooms**       | **0.31**    | Mild correlation — price increases somewhat with bathroom count, but not linearly.                                                                        |
| **price_log ↔ bedrooms**        | **0.21**    | Weak correlation — price doesn’t increase as predictably with bedroom count, possibly because extra bedrooms add less marginal value than square footage. |


## Skewnesss by State

In [ ]:
state_groups = cleaned_subset_df.groupby("state")
for state, group in state_groups:
    numeric_df = group[["bathrooms", "bedrooms", "price_log", "square_feet_log"]]
    skew_per_state = numeric_df.skew()
    print(f"{state}\n{skew_per_state}\n")

## Correlation by State

In [ ]:
correlation_by_state = {}
for state, group in state_groups:
    numeric_df = group[["bathrooms", "bedrooms", "price_log", "square_feet_log"]]
    correlation_by_state[state] = numeric_df.corr()

In [ ]:
n_states = len(correlation_by_state.keys())
# Grid size (fits 51)
rows = 8
cols = 7
fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
axes = axes.flatten()
for idx, state in enumerate(correlation_by_state.keys()):
    ax = axes[idx]
    corr = correlation_by_state[state]

    sns.heatmap(
        corr,
        ax=ax,
        cmap="mako",
        square=True,
        xticklabels=True,
        yticklabels=True,
        annot=True,
        cbar=False
    )

    ax.set_title(state, fontsize=14)
    ax.tick_params(axis='x', labelrotation=45, labelsize=10)
    ax.tick_params(axis='y', labelrotation=45, labelsize=10)

# Hide any unused subplot cells
for j in range(len(correlation_by_state.keys()), len(axes)):
    axes[j].axis("off")

fig.suptitle("Correlation Matrices by State", fontsize=18)
plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.99))
plt.show()

# K Means Clustering

In [ ]:
clusterer = KMeans(n_clusters=30, random_state=42)
cleaned_subset_df['geo_cluster'] = clusterer.fit_predict(cleaned_subset_df[['latitude', 'longitude']])

In [ ]:
plt.figure(figsize=(8, 8))

ax = sns.scatterplot(
    data=cleaned_subset_df,
    x="longitude",
    y="latitude",
    hue="geo_cluster",
    palette="tab20",
    s=30
)

handles, labels = ax.get_legend_handles_labels()
plt.legend(
    handles=handles,
    labels=labels,
    title="Cluster",
    ncol=3,
    bbox_to_anchor=(1,1),
    loc="upper left"
)

plt.show()

In [ ]:
X = cleaned_subset_df[['square_feet_log', 'bedrooms', 'bathrooms', 'state', 'geo_cluster', 'cityname']]
y = cleaned_subset_df['price_log']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

preprocessor = ColumnTransformer([
    ("square_feet_log_poly", PolynomialFeatures(degree=2, include_bias=False), ['square_feet_log']),
    ('structura_linear', 'passthrough', ['bedrooms', 'bathrooms']),
    ('city_encoded', OneHotEncoder(handle_unknown='ignore'), ['cityname']),
    ('neighborhood_cluster_encoded', OneHotEncoder(handle_unknown='ignore'), ['geo_cluster'])
])

model = Pipeline([
    ('preprocess', preprocessor),
    ('linreg', LinearRegression()),
])


model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# ==========================
# 7. Metrics helper (includes Adjusted R²)
# ==========================
def adjusted_r2(y_true, y_pred, n_features: int) -> float:
    """Compute Adjusted R² given predictions and number of original features."""
    r2 = r2_score(y_true, y_pred)
    n = len(y_true)
    p = n_features
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)


def print_regression_metrics(split_name, y_true, y_pred, n_features: int):
    r2 = r2_score(y_true, y_pred)
    adj_r2 = adjusted_r2(y_true, y_pred, n_features)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    med_ae = median_absolute_error(y_true, y_pred)

    print(f"=== {split_name} Metrics ===")
    print(f"R²:              {r2:.2f}")
    print(f"Adjusted R²:     {adj_r2:.2f}")
    print(f"MSE:             {mse:.2f}")
    print(f"RMSE:            {rmse:.2f}")
    print(f"MAE:             {mae:.2f}")
    print(f"MAPE:            {mape:.2f}")
    print(f"Median Abs Err:  {med_ae:.2f}")
    print()


# ==========================
# 8. Interpret real-world error from log-space metrics
# ==========================
def interpret_log_error_metrics(split_name, y_true, y_pred):
    """
    Interpret log-space error metrics in real price terms.
    Assumes y_true and y_pred are log(price).
    """
    rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))
    mae_log = mean_absolute_error(y_true, y_pred)
    med_ae_log = median_absolute_error(y_true, y_pred)

    # Convert log errors to multiplicative factors
    rmse_factor = np.exp(rmse_log)
    mae_factor = np.exp(mae_log)
    med_factor = np.exp(med_ae_log)

    rmse_pct = (rmse_factor - 1.0) * 100.0
    mae_pct = (mae_factor - 1.0) * 100.0
    med_pct = (med_factor - 1.0) * 100.0

    print(f"=== {split_name} Real-World Error Interpretation ===")
    print(f"Typical error (RMSE):    x {rmse_factor:.3f}  (predictions are typically off by ~{rmse_pct:.1f}% in true price)")
    print(f"Average error (MAE):     x {mae_factor:.3f}  (On average, predictions are off by ~{mae_pct:.1f}% in true price)")
    print(f"Typical core error (Median AE): x {med_factor:.3f}  (For half the homes, predictions are off within ~{med_pct:.1f}% in true price)")
    print()


# number of ORIGINAL features (bathrooms, bedrooms, sqft_log, state)
n_features = model.named_steps["preprocess"].transform(X_train).shape[1]

print_regression_metrics("Train", y_train, y_train_pred, n_features)
print_regression_metrics("Test",  y_test,  y_test_pred,  n_features)

interpret_log_error_metrics("Train", y_train, y_train_pred)
interpret_log_error_metrics("Test",  y_test,  y_test_pred)


| Metric               | Train Value | Test Value | Interpretation                                                                                                                   |
|----------------------|-------------|------------|----------------------------------------------------------------------------------------------------------------------------------|
| **R²**               | 0.78        | 0.76       | % of variance explained. Model captures ~78% of training variation and ~76% of unseen variation **strong fit, low overfitting**. |
| **MSE**              | 0.04        | 0.04       | Average squared error. Test MSE only slightly higher **good generalization**.                                                    |
| **RMSE**             | 0.20        | 0.21       | Typical prediction error magnitude, ~20–21% deviation in normalized/log scale.                                                   |
| **MAE**              | 0.15        | 0.15       | Average absolute error, predictions off by ~15%.                                                                                 |
| **MAPE**             | 0.02        | 0.02       | Mean Absolute Percentage Error ~2% if target is scaled, or ~2% error in/log space. Very low **consistent accuracy**.             |
| **Median Abs Error** | 0.11        | 0.11       | Half of predictions are within **11% error**. Indicates tight, stable core accuracy.                                             |


In [ ]:
train_resid = y_train - y_train_pred
test_resid  = y_test  - y_test_pred

In [ ]:
# Build DataFrames for easy plotting
train_df = pandas.DataFrame({
    "Fitted": y_train_pred,
    "Residual": train_resid,
    "Split": "Train"
})

test_df = pandas.DataFrame({
    "Fitted": y_test_pred,
    "Residual": test_resid,
    "Split": "Test"
})

resid_df = pandas.concat([train_df, test_df], ignore_index=True)

# Plot
plt.figure(figsize=(8, 8))
sns.scatterplot(
    data=resid_df,
    x="Fitted",
    y="Residual",
    hue="Split",
    alpha=0.5
)

plt.axhline(0, linestyle="--", color="black", linewidth=1)
plt.title("Residuals vs Fitted")
plt.xlabel("Fitted")
plt.ylabel("Residual")
plt.tight_layout()
plt.show()

In [ ]:
# DO NOT DELETE MIGHT NEED
# import json
# s = cleaned_uci_df['bathrooms'].explode()
# global_counts = s.value_counts().to_dict()
# global_counts
# print(f'BAD_DATA: {json.dumps(BAD_DATA['cityname'], indent=2)}')  # CHANGE COL
# uci_df["state_full"] = uci_df["state"].str.upper().map(STATE_MAP)
# print(uci_df.shape)
# uci_df.dropna(subset=["state_full"], inplace=True)
# uci_df.shape

# Simple Linear Regression

In [ ]:
lin_regression_residuals = {}
for state, group in state_groups:
    # choose numeric columns only
    X = group[["bathrooms", "bedrooms", "square_feet_log"]]
    y = group[['price_log']]
    model = LinearRegression()
    model.fit(X, y)
    # Predictions and residuals
    y_pred = model.predict(X)
    residuals = y - y_pred
    lin_regression_residuals[state] = y_pred, residuals

In [ ]:
# Grid size (fits 51)
rows = 8
cols = 7
fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
axes = axes.flatten()
for idx, state in enumerate(lin_regression_residuals.keys()):
    ax = axes[idx]
    y_pred, residuals = lin_regression_residuals[state]
    ax.scatter(y_pred, residuals)
    ax.axhline(0, color='red', linestyle='--', linewidth=2)
    ax.set_title(state, fontsize=14)
    ax.set_xlabel('Predicted price_log', fontsize=10)
    ax.set_ylabel('Residuals', fontsize=10)
    ax.set_title(state, fontsize=14)
# Hide any unused subplot cells
for j in range(len(lin_regression_residuals.keys()), len(axes)):
    axes[j].axis("off")
plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.99))
plt.show()